In [81]:
import pandas as pd
import numpy as np
import sklearn as sk
import sklearn.impute as skpute
import sklearn.model_selection as skmod
import sklearn.preprocessing as skpre
import sklearn.linear_model as sklin
import sklearn.metrics as skmet
import matplotlib.pyplot as plt
%matplotlib inline

folder = 'Data/'
diabetes = pd.read_csv(folder+'diabetes.csv')

diabetes.info()
diabetes.describe()

diabetes['BloodPressure'].unique()


diabetes['Age'].unique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


array([50, 31, 32, 21, 33, 30, 26, 29, 53, 54, 34, 57, 59, 51, 27, 41, 43,
       22, 38, 60, 28, 45, 35, 46, 56, 37, 48, 40, 25, 24, 58, 42, 44, 39,
       36, 23, 61, 69, 62, 55, 65, 47, 52, 66, 49, 63, 67, 72, 81, 64, 70,
       68], dtype=int64)

In [82]:
colnames = diabetes.columns

for col in colnames:
    avg,std = diabetes[col].mean(),diabetes[col].std()
    print(f'{col} Avg: {avg}, Std: {std}')
    print(diabetes[(diabetes[col] > avg + 3*std) | (diabetes[col] < avg - 3*std)])



Pregnancies Avg: 3.8450520833333335, Std: 3.3695780626988694
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
88            15      136             70             32      110  37.1   
159           17      163             72             41      114  40.9   
298           14      100             78             25      184  36.6   
455           14      175             62             30        0  33.6   

     DiabetesPedigreeFunction  Age  Outcome  
88                      0.153   43        1  
159                     0.817   47        1  
298                     0.412   46        1  
455                     0.212   38        1  
Glucose Avg: 120.89453125, Std: 31.97261819513622
     Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
75             1        0             48             20        0  24.7   
182            1        0             74             20       23  27.7   
342            1        0             68             35        0  

In [83]:
diabetes.info()
diabetes[diabetes['BMI'] == 0]

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               768 non-null    int64  
 1   Glucose                   768 non-null    int64  
 2   BloodPressure             768 non-null    int64  
 3   SkinThickness             768 non-null    int64  
 4   Insulin                   768 non-null    int64  
 5   BMI                       768 non-null    float64
 6   DiabetesPedigreeFunction  768 non-null    float64
 7   Age                       768 non-null    int64  
 8   Outcome                   768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
9,8,125,96,0,0,0.0,0.232,54,1
49,7,105,0,0,0,0.0,0.305,24,0
60,2,84,0,0,0,0.0,0.304,21,0
81,2,74,0,0,0,0.0,0.102,22,0
145,0,102,75,23,0,0.0,0.572,21,0
371,0,118,64,23,89,0.0,1.731,21,0
426,0,94,0,0,0,0.0,0.256,25,0
494,3,80,0,0,0,0.0,0.174,22,0
522,6,114,0,0,0,0.0,0.189,26,0
684,5,136,82,0,0,0.0,0.640,69,0


In [84]:
discrete_columns = colnames[1:4]

DiscreteImputer = skpute.SimpleImputer(missing_values=0,strategy='median')
discrete_imputed_df = pd.DataFrame(DiscreteImputer.fit_transform(diabetes[discrete_columns]), columns=discrete_columns)

continuous_columns = colnames[5]
ContinuousImputer = skpute.SimpleImputer(missing_values=0,strategy='mean')
continuous_imputed_df = pd.DataFrame(ContinuousImputer.fit_transform(diabetes[continuous_columns].values.reshape(-1,1)),columns=[continuous_columns])

for col in discrete_columns:
    diabetes[col] = discrete_imputed_df[col]

diabetes['BMI'] = continuous_imputed_df

diabetes[diabetes['BloodPressure'] == 0]

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome


In [85]:
X,y = skpre.scale(diabetes.drop(columns=['Outcome'])), diabetes['Outcome']
X_train,X_test,y_train,y_test = skmod.train_test_split(X,y,test_size=.25,random_state=0)

mod = sklin.LogisticRegressionCV().fit(X_train,y_train)

In [90]:
y_predict = mod.predict(X_test)

confusmat = skmet.confusion_matrix(y_test,y_predict)
print(confusmat)
normal = confusmat/len(y_test)
print(normal)


[[117  13]
 [ 26  36]]
[[0.609375   0.06770833]
 [0.13541667 0.1875    ]]


In [ ]:
matthews = skmet.matthews_corrcoef(y_test,y_predict)
print(matthews)

0.5154968748198281
